# Baseline Tax Model

This shows steps to numerically linearize the "Baseline" model we have been using in class

In [44]:
import numpy as np
import scipy.optimize as opt

from collections import namedtuple

Will store parameters of model in a `namedtuple`

In [10]:
params = ["a", "alpha", "beta", "delta", "gamma", "rho", "sigma", "tau", "xi"]
BaselineModel = namedtuple("BaselineModel", params)

In [11]:
m = BaselineModel(0.50, 0.40, 0.98, 0.10, 2.50, 0.90, 0.02, 0.05, 1.5)

Notice, `namedtuple`s allow you to identify parameters by name which helps prevent errors

In [12]:
m.alpha, m.beta

(0.4, 0.98)

## Utility and Production Functions

DRY!!!

\begin{align*}
  u(c, l) &= \frac{c^{1 - \gamma} - 1}{1 - \gamma} + a \frac{(1 - l)^{1 - \xi} - 1}{1 - \xi} \\
  f(k, l, z) &= k^{\alpha} (l \exp(z))^{1 - \alpha}
\end{align*}

In [68]:
def u(m, c, l):
    uc = (c**(1.0 - m.gamma) - 1.0) / (1.0 - m.gamma)
    ul = m.a * ((1.0 - l)**(1.0 - m.xi) - 1.0) / (1.0 - m.xi)
    return uc + ul

def du_dc(m, c, l):
    return c**(-m.gamma)

def du_dl(m, c, l):
    return - m.a * (1.0 - l)**(-m.xi)

def f(m, k, l, z):
    return k**m.alpha * (l * np.exp(z))**(1.0 - m.alpha)

def df_dk(m, k, l, z):
    return m.alpha * (l/k * np.exp(z))**(1.0 - m.alpha)

def df_dl(m, k, l, z):
    return (1.0 - m.alpha) * np.exp(z)**(1.0 - m.alpha) * (k / l)**m.alpha


## Characteristic Equations

DRY!!!

\begin{align*}
  c_t &= (1 - \tau) (w_t l_t + (r_t - \delta) k_t) + T_t - k_{t+1} \\
  u_c(c_t, l_t) &= \beta E \left[ u_c(c_{t+1}, l_{t+1}) (r_{t+1} - \delta) (1 - \tau) + 1) \right] \\
  -u_l(c_t, l_t) &= u_c(c_t, l_t) w_t (1 - \tau) \\
  r_t &= f_K(k_t, l_t, z_t) \\
  w_t &= f_L(k_t, l_t, z_t) \\
  T_t &= \tau [w_t l_t + (r - \delta) k_t] \\
  z_{t+1} &= \rho z_t + \sigma \varepsilon_{t+1}
\end{align*}

In [ ]:
def Definitions(m, ktp1, kt, lt, zt):
    wt = df_dl(m, kt, lt, zt)
    rt = df_dk(m, kt, lt, zt)
    Tt = m.tau * (wt*lt + (rt - m.delta)*kt)
    ct = (1 - m.tau) * (wt*lt + (rt - m.delta)*kt) + kt + Tt - ktp1

    ct = max(ct, 1e-8)

    return ct, rt, Tt, wt


def CharacteristicEquations(m, ktp2, ltp1, ktp1, lt, kt, ltm1, ztp1, zt):
    ct, rt, Tt, wt = Definitions(m, ktp1, kt, lt, zt)
    ctp1, rtp1, Ttp1, wtp1 = Definitions(m, ktp2, ktp1, ltp1, ztp1)
    rr_tp1 = (rtp1 - m.delta)*(1 - m.tau) + 1.0

    intertemporal_ee = du_dc(m, ct, lt) - m.beta*du_dc(m, ctp1, ltp1)*rr_tp1
    intratemporal_ee = du_dl(m, ct, lt) + du_dc(m, ct, lt)*wt*(1.0 - m.tau)

    return np.array([intertemporal_ee, intratemporal_ee])

## Steady state

To find steady state, find $\bar{k}$ and $\bar{l}$ such that $\Gamma(\bar{X}, \bar{Z}) = 0$

In [79]:
def solve_ss(m):
    solve_me = lambda x: CharacteristicEquations(m, x[0], x[1], x[0], x[1], x[0], x[1], 0.0, 0.0)
    sol = opt.root(solve_me, np.array([0.5, 0.85]))

    return sol.x

In [81]:
kbar, lbar = solve_ss(m)

cbar, rbar, Tbar, wbar = Definitions(m, kbar, kbar, lbar, 0.0)

## Derivatives

In [90]:
CharacteristicEquations(m, *[kbar, lbar, kbar, lbar, kbar, lbar, 0.0, 0.0])

array([ 7.26307903e-13, -2.11608508e-13])

In [97]:
def derivative(f, x, xtilde):
    return (f(x) - f(xtilde)) / np.sum(x - xtilde)


def linearize(m, kbar, lbar, eps=1e-6):
    # Compute ss gamma
    x = np.array([kbar, lbar, kbar, lbar, kbar, lbar, 0.0, 0.0])
    Gamma_ss = CharacteristicEquations(m, kbar, lbar, kbar, lbar, kbar, lbar, 0.0, 0.0)

    # Allocate space for derivative matrix
    neq = len(Gamma_ss)
    Dmat = np.empty((neq, 8))

    CEsplat = lambda x: CharacteristicEquations(m, *list(x))
    for i in range(8):
        xtilde = np.array([kbar, lbar, kbar, lbar, kbar, lbar, 0.0, 0.0])
        xtilde[i] += eps
        Dmat[:, i] = derivative(CEsplat, x, xtilde)

    F = Dmat[:, :2]
    G = Dmat[:, 2:4]
    H = Dmat[:, 4:6]
    L = Dmat[:, 6]
    M = Dmat[:, 7]

    return F, G, H, L, M


In [99]:
F, G, H, L, M = linearize(m, kbar, lbar)


In [100]:
F

array([[-4.22623769,  5.44191867],
       [-0.        , -0.        ]])

In [103]:
G

array([[  8.5666152 ,  -5.61221554],
       [  5.33163184, -14.89888287]])

In [104]:
H

array([[-4.31700912, -0.        ],
       [-5.27237248, -0.        ]])

In [105]:
L

array([ 3.1551835, -0.       ])

In [106]:
M

array([-3.25392039, -3.00365571])